# Access DB2 Warehouse on Cloud and explore the data with Python

This notebook shows how to access DB2 Warehouse on Cloud and explore the loaded data set using Python.


## Table of contents

1. [Info](#Info)
2. [Create Version](#Version)
3. [Setup](#Setup)
4. [Import the *ibmdbpy* Python library](#Import-the-ibmdbpy-Python-library)
5. [Insert the database connection credentials](#Insert-the-database-connection-credentials)
6. [Create the database connection](#Create-the-database-connection)
7. [Use a dataframe to read and manipulate tables](#Use-a-dataframe-to-read-and-manipulate-tables)
8. [Explore the supply chain data](#Explore-the-supply-chain-data)
9. [Verify Spark version and existence of Spark](#Verify-Spark-version-and-existence-of-Spark)
10. [Connect to the database and read in our data into a Spark DataFrame](#Connect-to-the-database-and-read-in-our-data-into-a-Spark-DataFrame)
11. [Identify our labels and transform](#Identify-our-labels-and-transform)
12. [Use StringIndexer to convert all String columns to columns of Double values](#Use-StringIndexer-to-convert-all-String-columns-to-columns-of-Double-values)
13. [Feature Engineering](#Feature-Engineering)
14. [Write the new table to DB2 Warehouse](#Write-the-new-table-to-DB2-Warehouse)
15. [Create Version](#Create-Version)
16. [Download notebook](#Download-notebook)
17. [Resources](#Resources)
18. [Get Help](#Get-Help)
19. [Summary](#Summary)
20. [Take it farther](#Take-it-farther)

<a id=\"Info\"></a>
## Info

Select the Info icon to display general information about the notebook.

![](https://raw.githubusercontent.com/jpatter/DSX/master/Lab-1/images/info-button.png)

You can change the notebook name or add a description here.   You can also flag that the notebook can run with your privileges (Trusted Notebook). If you run all cells of the notebook it is also considered trusted.

![](https://s10.postimg.org/sdbt8xkm1/Trusted_Notebook.png)

<a id=\"Version\"></a>
## Create Version

Save a version of the notebook by selecting File > Save Version

![](https://raw.githubusercontent.com/jpatter/DSX/master/Lab-1/images/FileOptions.PNG)

or by selecting the Versions icon. 

![](https://raw.githubusercontent.com/jpatter/DSX/master/Lab-1/images/versions-button.png)

You can have up to ten (10) versions of a notebook. Notebook versions are saved in a FIFO manner.

<a id=\"Setup\"></a>
## Setup

Before beginning this lab, you will need a *DB2 Warehouse on Cloud* instance. DB2 Warehouse is a fully managed cloud data warehouse, purpose-built for analytics. It offers massively parallel processing (MPP) scale and compatibility with a wide range of business intelligence (BI) tools.

1. __RIGHT-CLICK__ [this link](https://github.com/cerebralace/LMDSX/blob/master/Lab-1/DQIM-SupplyChain-Data.csv) and select __Save Link As...__ to save the `DQIM-SupplyChain-Data.csv` file to your laptop.
2. Download the [DB2 Warehouse setup instructions](https://github.com/cerebralace/LMDSX/blob/master/Lab-1/DB2Warehouse_DSXPOT.pdf).  You need to download the PDF in order for the links to work.
3. Open up the PDF file that you've just downloaded in the previous step and follow the instructions there to setup DB2 Warehouse, load the supply chain data and create a DSX data connection.

[![](https://raw.githubusercontent.com/jpatter/DSX/master/Lab-1/images/DB2Warehouse.png)](https://console.ng.bluemix.net/catalog/services/dashdb)

<a id=\"Import-the-ibmdbpy-Python-library\"></a>
## Import the *ibmdbpy* Python library

Python support for DB2 Warehouse is provided by the [ibmdbpy Python library](https://pypi.python.org/pypi/ibmdbpy). Connecting to DB2 Warehouse is also enabled by a DB2 driver, libdb2.so.  The ibmdbpy project provides a Python interface for data manipulation and access to in-database algorithms in IBM DB2 Warehouse. It accelerates Python analytics by seamlessly pushing operations written in Python into the underlying database for execution, thereby benefitting from in-database performance-enhancing features, such as columnar storage and parallel processing.

### Setup a *JDBC* connection

The JDBC Connection is based on a Java virtual machine. From the ibmdbpy library you can use JDBC to connect to a remote DB2 Warehouse/DB2 instance. To be able to use JDBC to connect, we need to import the *JayDeBeApi* package.

To run a cell, click on the cell and click the run button in the toolbar or press __SHIFT-ENTER__.

Run the following commands to install and load the `JayDeBeApi` package and the `ibmdbpy` library into your notebook:

In [1]:
import jaydebeapi
from ibmdbpy import IdaDataBase
from ibmdbpy import IdaDataFrame

In [2]:
import pixiedust

Pixiedust database opened successfully


<a id=\"Insert-the-database-connection-credentials\"></a>
## Insert the database connection credentials

Click on the cell below, then on the notebook toolbar, click the box of 1's and 0's, find your database connection and click the __Insert to code__ link under the connection name to have a `credentials_1` dictionary added to the notebook.

![](https://raw.githubusercontent.com/jpatter/DSX/master/Lab-1/images/connections-button.png)

__Note__: When this dictionary is inserted, it might have a different name than `credentials_1`.  If so change the dictionary name to `credentials_1` so that the rest of the notebook will work.

Connecting to DB2 Warehouse requires the following information which are provided by the credentials dictionary inserted:
* Database name
* Host DNS name or IP address
* Host port
* Connection protocol
* User ID
* User password

The information `credentials_1` will be used to build a connection string in a subsequent step.

In [3]:
# insert connection here
# @hidden_cell
credentials_1 = {
  'database':'BLUDB',
  'password':"""Tgl4_RiQA8_h""",
  'sg_service_url':'https://sgmanager.ng.bluemix.net',
  'host':'dashdb-entry-yp-dal09-07.services.dal.bluemix.net',
  'username':'dash10135'
}

<a id=\"Create-the-database-connection\"></a>
## Create the database connection

The following code snippet creates a connection string `connection_string` and uses the `connection_string` to create a database connection object:

In [4]:
connection_string='DASHDB;Database={database};Hostname={host};Port=50000;PROTOCOL=TCPIP;UID={username};PWD={password}'.format(**credentials_1)
idadb = IdaDataBase(dsn=connection_string)

<a id=\"Use-dataframe-to-read-and-manipulate-tables\"></a>
## Use a dataframe to read and manipulate tables

You can now use the connection object `idadb` to query the database:

In [5]:
df = idadb.show_tables(show_all = True)
df.head(10)

,TABSCHEMA,TABNAME,OWNER,TYPE
0,DASH10135,SUPPLYCHAIN,DASH10135,T
1,GOSALES,BRANCH,DB2INST1,T
2,GOSALES,CONVERSION_RATE,DB2INST1,T
3,GOSALES,COUNTRY,DB2INST1,T
4,GOSALES,CURRENCY_LOOKUP,DB2INST1,T
5,GOSALES,EURO_CONVERSION,DB2INST1,T
6,GOSALES,INVENTORY_LEVELS,DB2INST1,T
7,GOSALES,ORDER_DETAILS,DB2INST1,T
8,GOSALES,ORDER_HEADER,DB2INST1,T
9,GOSALES,ORDER_METHOD,DB2INST1,T


In [6]:
idadb.exists_table_or_view('GOSALESDW.EMP_EXPENSE_FACT')

True

Using our previously opened IdaDataBase instance named ‘idadb’, we can open one or several IdaDataFrame objects. They behave like pointers to remote tables.

Let us open the *EMP_EXPENSE_FACT* data set, assuming it is stored in the database under the name ‘GOSALESDW.EMP_EXPENSE_FACT’. The following cell assigns the dataset to a IdaDataFrame. IdaDataFrame copies the Pandas interface for DataFrame objects to ensure intuitive interaction for end-users.

The [Pandas data analysis library](http://pandas.pydata.org/) provides high-performance, easy-to-use data structures and data analysis tools for the Python programming language. Pandas allows easy processing and manipulation of tabular data, so it is a perfect fit for data extracted from relational databases.

In [7]:
idadf = IdaDataFrame(idadb, 'GOSALESDW.EMP_EXPENSE_FACT')

### Automatic creation of dataframes for tables

In [8]:
idadf.dtypes

,TYPENAME
DAY_KEY,INTEGER
ORGANIZATION_KEY,INTEGER
POSITION_KEY,INTEGER
EMPLOYEE_KEY,INTEGER
EXPENSE_TYPE_KEY,INTEGER
ACCOUNT_KEY,INTEGER
EXPENSE_UNIT_QUANTITY,DOUBLE
EXPENSE_TOTAL,DECIMAL


You can very easily explore the data in the IdaDataFrame by using built in functions.

Use IdaDataFrame.head to get the first n records of your data set (default 5):

In [9]:
idadf.head()

,DAY_KEY,ORGANIZATION_KEY,POSITION_KEY,EMPLOYEE_KEY,EXPENSE_TYPE_KEY,ACCOUNT_KEY,EXPENSE_UNIT_QUANTITY,EXPENSE_TOTAL
0,20100131,11106,43638,4011,2131,8049,165.000,3378.21
1,20100131,11106,43638,4011,2122,8054,0.110,554.72
2,20100131,11106,43638,4011,2120,8052,0.080,403.44
3,20100131,11106,43638,4011,2130,8050,0.005,1664.70
4,20100131,11106,43638,4011,2124,8056,0.030,151.29


Use IdaDataFrame.tail to get the last n records of your data set (default 5):

In [10]:
idadf.tail(10)

,DAY_KEY,ORGANIZATION_KEY,POSITION_KEY,EMPLOYEE_KEY,EXPENSE_TYPE_KEY,ACCOUNT_KEY,EXPENSE_UNIT_QUANTITY,EXPENSE_TOTAL
127974,20130731,11203,43628,4712,2122,8054,0.11,453.75
127975,20130731,11186,43627,4768,2122,8054,0.11,273.29
127976,20130731,11203,43628,4712,2124,8056,0.03,123.75
127977,20130731,11186,43627,4768,2124,8056,0.03,74.53
127978,20130731,11203,43628,4712,2131,8049,165.00,4125.00
127979,20130731,11190,43602,4958,2120,8052,0.08,907.69
127980,20130731,11190,43602,4958,2122,8054,0.11,1248.08
127981,20130731,11190,43602,4958,2124,8056,0.03,340.38
127982,20130731,11190,43602,4958,2131,8049,165.00,11346.15
127983,20130731,11190,43602,4958,2134,8050,37.50,13500.00


__Note__: Because DB2 Warehouse operates on a distributed system, the order of rows using IdaDataFrame.head and IdaDataFrame.tail is not guaranteed unless the table is sorted (using an ‘ORDER BY’ clause) or a column is declared as index for the IdaDataFrame (parameter/attribute indexer)

IdaDataFrame also implements most attributes that are available in a pandas DataFrame, here is an example of getting the _shape_ of the data frame (rows, columns):

In [11]:
idadf.shape

(127984, 8)

In [12]:
idadf.columns

Index([u'DAY_KEY', u'ORGANIZATION_KEY', u'POSITION_KEY', u'EMPLOYEE_KEY',
       u'EXPENSE_TYPE_KEY', u'ACCOUNT_KEY', u'EXPENSE_UNIT_QUANTITY',
       u'EXPENSE_TOTAL'],
      dtype='object')

Several standard statistics functions from the pandas interface are also available for IdaDataFrame. You can use the __TAB__ key after the `.` to see the methods and attributes available in an object in a pulldown menu.

For example, let us calculate various summary statistics for the data set by using the `describe()` method for the IdaDataFrame instance:

In [13]:
idadf.describe()

,DAY_KEY,ORGANIZATION_KEY,POSITION_KEY,EMPLOYEE_KEY,EXPENSE_TYPE_KEY,ACCOUNT_KEY,EXPENSE_UNIT_QUANTITY,EXPENSE_TOTAL
count,1.279840e+05,127984.000000,127984.000000,127984.000000,127984.000000,127984.000000,127984.000000,127984.000000
mean,2.011514e+07,11157.820524,43627.441797,4489.672811,2120.315289,8052.091636,31.291244,1318.107960
std,1.036554e+04,31.272648,12.175158,298.987628,9.386336,2.532779,57.717638,2078.720318
min,2.010013e+07,11101.000000,43601.000000,4001.000000,2101.000000,8049.000000,-15.000000,-658.010000
25%,2.011013e+07,11131.000000,43617.000000,4216.000000,2120.000000,8050.000000,0.080000,211.280000
50%,2.011123e+07,11166.000000,43630.000000,4480.000000,2122.000000,8052.000000,0.110000,438.045000
75%,2.012103e+07,11182.000000,43637.000000,4763.000000,2124.000000,8054.000000,16.250000,1598.345000
max,2.013073e+07,11204.000000,43656.000000,4972.000000,2137.000000,8056.000000,172.500000,28776.920000


<a id=\"Explore-the-supply-chain-data\"></a>
## Explore the supply chain data

__Time to apply what you've learned!__
Complete the tasks that are commented in the code blocks. If you have any questions, feel free to ask.

### Create an ibmdbpy dataframe called `supplychain_df` and set it to the name of the supply chain table in your DB2 Warehouse 

Select the data cell you wish the code to be added to

Select the Find and Add Data icon

Select Connections

Select Insert to Code for the connection you wish to use (there should only be one)

Select Insert ibmdbpy IdaDataframe

Select the Schema you used to load the data (this schema should start with DASH)

Select the Table (there should only be one)

Select Insert Code

Rename the result to supplychain_df

In [14]:
# insert dataframe here
from ibmdbpy import IdaDataBase, IdaDataFrame

# @hidden_cell
# This connection object is used to access your data and contains your credentials.
# You might want to remove those credentials before you share your notebook.
idadb_4124e05115af4b34a08fb99170568e2b = IdaDataBase(dsn='DASHDB;Database=BLUDB;Hostname=dashdb-entry-yp-dal09-07.services.dal.bluemix.net;Port=50000;PROTOCOL=TCPIP;UID=dash10135;PWD=Tgl4_RiQA8_h')

supplychain_df = IdaDataFrame(idadb_4124e05115af4b34a08fb99170568e2b, 'DASH10135.SUPPLYCHAIN')
supplychain_df.head()

# You can close the database connection with the following code. Please keep the comment line with the @hidden_cell tag,
# because the close function displays parts of the credentials.
# @hidden_cell
# idadb_4124e05115af4b34a08fb99170568e2b.close()
# To learn more about the ibmdby package, please read the documentation: http://pythonhosted.org/ibmdbpy/


,Report #,Title,Report Type,Category,Severity,Date Discovered,Date Reported,Date Answered,Date Resolved,LCN,Part Number,Serial Number
0,A2UN-1-100422411,A2UN-1-92251791 Post SPIR Update,DQIM,CAT II,High,11/27/17 1:55,11/27/17 15:13,0,0,Default,0,0
1,A2UN-1-100616351,Duplicate Inventory: 2AF:0076/13-5070 requires...,DQIM,CAT II,Low,11/28/17 18:38,11/28/17 20:04,1/8/18 20:08,1/8/18 20:08,A9941114,77454451-002,10136
2,A2UN-1-101732671,Holes for P5 fol1ing AARI MOD,DQIM,CAT I,Medium,12/6/17 13:00,12/7/17 16:53,12/18/17 20:40,12/18/17 20:40,ALIS,07142000-1,0
3,A2UN-1-103416581,Substitute of GUC,Supply Chain,CAT II,Low,12/21/17 11:47,12/22/17 19:06,0,0,ALIS,0,0
4,A2UN-1-14584147,Variances in EEL and ALIS 1.0.3,DQIM,CAT II,Low,2/11/13 15:20,2/11/13 20:21,8/28/14 20:33,8/28/14 8:33,DQIM,0,0


In [15]:
idadb.exists_table_or_view(supplychain_df.name)

# The output of this block should be:
# True

True

In [16]:
print(supplychain_df.shape)
supplychain_df.dtypes

(21884, 12)


,TYPENAME
Report #,VARCHAR
Title,VARCHAR
Report Type,VARCHAR
Category,VARCHAR
Severity,VARCHAR
Date Discovered,VARCHAR
Date Reported,VARCHAR
Date Answered,VARCHAR
Date Resolved,VARCHAR
LCN,VARCHAR


### Show the top 5 records from the table using a method of the IdaDataFrame

Type supplychain_df.head()

In [17]:
# enter answer here
supplychain_df.head()

,Report #,Title,Report Type,Category,Severity,Date Discovered,Date Reported,Date Answered,Date Resolved,LCN,Part Number,Serial Number
0,A2UN-1-100422411,A2UN-1-92251791 Post SPIR Update,DQIM,CAT II,High,11/27/17 1:55,11/27/17 15:13,0,0,Default,0,0
1,A2UN-1-100616351,Duplicate Inventory: 2AF:0076/13-5070 requires...,DQIM,CAT II,Low,11/28/17 18:38,11/28/17 20:04,1/8/18 20:08,1/8/18 20:08,A9941114,77454451-002,10136
2,A2UN-1-101732671,Holes for P5 fol1ing AARI MOD,DQIM,CAT I,Medium,12/6/17 13:00,12/7/17 16:53,12/18/17 20:40,12/18/17 20:40,ALIS,07142000-1,0
3,A2UN-1-103416581,Substitute of GUC,Supply Chain,CAT II,Low,12/21/17 11:47,12/22/17 19:06,0,0,ALIS,0,0
4,A2UN-1-14584147,Variances in EEL and ALIS 1.0.3,DQIM,CAT II,Low,2/11/13 15:20,2/11/13 20:21,8/28/14 20:33,8/28/14 8:33,DQIM,0,0


## Verify Spark version and existence of Spark

We will now use Spark in order to perform some exploratory analysis on the supply chain dataset.

In [18]:
print('The spark version is {}.'.format(spark.version))

The spark version is 2.1.2.


In [19]:
#Imports for Spark
from pyspark.ml.feature import StringIndexer, IndexToString
from pyspark.ml.feature import Bucketizer
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import year
from pyspark.sql.types import Row

# Imports for pixiedust
from pixiedust.display import *


## Connect to the database and read in our data into a Spark DataFrame


Select the <b>Find and Add Data</b> icon

![](https://raw.githubusercontent.com/jpatter/LMCO/master/Lab-1/images/connections-button.png)

Select the <b>Connections</b> view and then <b>Insert to code</b>.

![](https://raw.githubusercontent.com/jpatter/LMCO/master/Lab-1/images/InsertToCode.PNG)

Select <b>Insert SparkSession DataFrame</b> and select the schema (will start with DASH but will likely NOT be the same value you see in the image) and table (should only be one). Then select <b>Insert Code</b>.

Rename the result to <b>supplychain_df</b> to ensure compliance with the following cells.

In [20]:
# Insert SparkSession DataFrame here
# make CERTAIN to rename to supplychain_df
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

# @hidden_cell
# The following code is used to access your data and contains your credentials.
# You might want to remove those credentials before you share your notebook.

properties_4124e05115af4b34a08fb99170568e2b = {
    'driver': 'com.ibm.db2.jcc.DB2Driver',
    'jdbcurl': 'jdbc:db2://dashdb-entry-yp-dal09-07.services.dal.bluemix.net:50000/BLUDB',
    'user': 'dash10135',
    'password': 'Tgl4_RiQA8_h'
}

supplychain_df = spark.read.jdbc(properties_4124e05115af4b34a08fb99170568e2b['jdbcurl'], table='DASH10135.SUPPLYCHAIN', properties=properties_4124e05115af4b34a08fb99170568e2b)
supplychain_df.show(5)

+--------------------+--------------------+-----------+--------+--------+---------------+-------------+-------------+-------------+--------+------------------+-------------+
|            Report #|               Title|Report Type|Category|Severity|Date Discovered|Date Reported|Date Answered|Date Resolved|     LCN|       Part Number|Serial Number|
+--------------------+--------------------+-----------+--------+--------+---------------+-------------+-------------+-------------+--------+------------------+-------------+
|S022UNUS01-1-1631...|AN1 - Need PN loa...|       DQIM|  CAT II|    High|  3/17/14 19:17|3/17/14 19:28|5/14/14 15:41| 5/21/14 8:24|A3344011|    2FCH73102-0003|            0|
|S022UNUS01-1-1631...|Need Station 3 an...|       DQIM|  CAT II|    High|  3/17/14 20:01|3/17/14 20:38|3/25/14 15:00| 3/26/14 3:54|E9400030|    2WSH53473-0004|            0|
|S022UNUS01-1-1634...|AF21 - Request au...|       DQIM|   CAT I|     Low|  3/20/14 10:51|3/20/14 11:27|3/25/14 12:19| 3/25/14 6:23

## Identify our labels and transform

We will use the 'SEVERITY' column as a label for training a machine learning model (in Lab 3).  Each discrepancy in the supply chain process has been marked according to severity level. 

Since Watson Machine Learning requires that that the labels are of data type Double, we will convert all values in the 'SEVERITY' column to Double values using StringIndexer.

This should give us an extra column for Severity Level. Since a Spark dataframe is immutable, we will drop the Severity column of strings and rename the Severity column of doubles to "Severity." Then, we will declare the modified dataframe as a new dataframe - supplychain_df - and return the first row of the table.

In [21]:
labeledData = StringIndexer(inputCol="Severity", outputCol="SevIndex", handleInvalid="error").fit(supplychain_df).transform(supplychain_df)
supplychain_df = labeledData.drop("Severity").withColumnRenamed("SevIndex", "Severity")
supplychain_df.show(5)

+--------------------+--------------------+-----------+--------+---------------+-------------+--------------+-------------+--------+--------------+-------------+--------+
|            Report #|               Title|Report Type|Category|Date Discovered|Date Reported| Date Answered|Date Resolved|     LCN|   Part Number|Serial Number|Severity|
+--------------------+--------------------+-----------+--------+---------------+-------------+--------------+-------------+--------+--------------+-------------+--------+
|S002UNUS01-1-2348...|(MIL-Egress) Cano...|       DQIM|  CAT II|  8/18/14 12:18| 8/19/14 9:09|12/17/14 15:44|12/18/14 2:24|     A56|2FSH75201-0007|            0|     0.0|
|S002UNUS01-1-2348...|(SCM/PFF) Create ...|       DQIM|  CAT II|  8/18/14 14:15|8/18/14 14:46| 8/19/14 11:52| 8/29/14 3:39|A4232080|  359510-02-02|            0|     1.0|
|S002UNUS01-1-2349...|(SCM/PFF) Create ...|       DQIM|  CAT II|  8/18/14 14:22|8/18/14 14:44| 8/19/14 11:53| 8/29/14 3:36|A4232080|  359510-02-0

## Use StringIndexer to convert all String columns to columns of Double values


In [22]:
#Use StringIndexer to convert the Report Type column
ReportData = StringIndexer(inputCol="Report Type", outputCol="RepIndex", handleInvalid="error").fit(supplychain_df).transform(supplychain_df)
supplychain_df = ReportData.drop("Report Type").withColumnRenamed("RepIndex", "Report_Type")

#Use StringIndexer to convert the Category column
CategoryData = StringIndexer(inputCol="Category", outputCol="CatIndex", handleInvalid="error").fit(supplychain_df).transform(supplychain_df)
supplychain_df = CategoryData.drop("Category").withColumnRenamed("CatIndex", "Category")

#Use StringIndexer to convert the LCN column to Double values
LCNData = StringIndexer(inputCol="LCN", outputCol="LCNIndex", handleInvalid="error").fit(supplychain_df).transform(supplychain_df)
supplychain_df = LCNData.drop("LCN").withColumnRenamed("LCNIndex", "Location")

#Print the first row of supplychain_df
supplychain_df.show(5)

+--------------------+--------------------+---------------+--------------+--------------+--------------+--------------------+-------------+--------+-----------+--------+--------+
|            Report #|               Title|Date Discovered| Date Reported| Date Answered| Date Resolved|         Part Number|Serial Number|Severity|Report_Type|Category|Location|
+--------------------+--------------------+---------------+--------------+--------------+--------------+--------------------+-------------+--------+-----------+--------+--------+
|S014UNUS01-1-2965...|OT&E EEL NOT CROS...| 11/17/16 19:12|11/18/16 16:52|11/18/16 20:54|  1/6/17 19:11|JSF P/N 2ZKH63010...|         5495|     0.0|        0.0|     1.0|     1.0|
|S014UNUS01-1-2965...|OTE SCM. MICAP EE...| 11/17/16 21:42|11/18/16 16:52|11/18/16 20:23| 2/15/17 17:29|JSF P/N 2ZKH85140...|         2092|     0.0|        0.0|     1.0|     1.0|
|S014UNUS01-1-2965...|LM MOD, AF23/5011...| 11/17/16 22:18|11/17/16 22:26| 11/18/16 0:51|11/21/16 13:18| 

Now, let's look at the data we have:

SEVERITY is at three different levels:

2.0 - HIGH

1.0 - MEDIUM

0.0 - LOW

Print the total number of severity statuses:

In [23]:
print('The total number of rows in the table is {}.'.format(supplychain_df.count()))
print('The number of rows labeled High is {}.'.format(supplychain_df.filter(supplychain_df['Severity'] == 2.0).count()))
print('The number of rows labeled Medium is {}.'.format(supplychain_df.filter(supplychain_df['Severity'] == 1.0).count()))
print('The number of rows labeled Low is {}.'.format(supplychain_df.filter(supplychain_df['Severity'] == 0.0).count()))

The total number of rows in the table is 21884.
The number of rows labeled High is 3057.
The number of rows labeled Medium is 3682.
The number of rows labeled Low is 15144.


## Feature Engineering

A feature refers to each element (or column) of the data that we will be using to create our machine learning model in Lab 3. We need to transform each one of our features into Double values and write a new table to our DB2 warehouse instance.

The features that we will be using to create our machine learning model will be Report_Type, Category and Location and the label that we will be predicting shall be the Severity level. By choosing these features, we are claiming that the severity of each discrepancy is dependent on the aforementioned features - the type of report, the category, and the location of the action request.

In [25]:
#Drop the columns that we will not be using to build the ML model: Report #, Date Discovered, Title, Date Reported, Date Answered, Date Resolved, LCN, Serial Number
modifiedsupplychain = supplychain_df.drop("Report #", "Date Discovered", "Title", "Date Reported", "Date Answered", "Date Resolved", "Serial Number", "Part Number")

#Use pixiedust to view the modified supply chain table
display(modifiedsupplychain)

Severity,Report_Type,Category,Location
0.0,0.0,0.0,15.0
0.0,0.0,0.0,4.0
0.0,0.0,0.0,29.0
0.0,0.0,0.0,29.0
0.0,0.0,0.0,4.0
0.0,0.0,0.0,4.0
2.0,0.0,0.0,35.0
0.0,0.0,0.0,309.0
0.0,0.0,0.0,128.0
0.0,0.0,0.0,975.0


## Write the new table to DB2 Warehouse

Insert the database connection credentials like we did earlier

Click on the cell below, then on the notebook toolbar, click the box of 1's and 0's, which is the <b>Find and Add Data</b> icon.

![](https://raw.githubusercontent.com/jpatter/LMCO/master/Lab-1/images/connections-button.png)

Find your database connection and click <b>Insert to code</b> then select the <b>Insert Credentials</b> link under the connection name to have a credentials dictionary added to the notebook. 

![](https://raw.githubusercontent.com/jpatter/LMCO/master/Lab-1/images/InsertToCode.PNG)


Connecting to DB2 Warehouse requires the following information which are provided by the credentials dictionary inserted:

Database name
Host DNS name or IP address
Host port
Connection protocol
User ID
User password

The information credentials_1 will be used to build a connection string in a subsequent step. Note: it is possible that the credentials may be named credentials_2, etc. If so, simply rename to credentials_1.

The @hidden_cell directive tells DSX not to export credentials when sharing.

In [26]:
# Add the credentials information here -- make CERTAIN it is called credentials_1
#If prompted for jdbcurl, the general format is 'jdbc:db2://<host>:50000/BLUDB'
# @hidden_cell
credentials_1 = {
  'database':'BLUDB',
  'password':"""Tgl4_RiQA8_h""",
  'sg_service_url':'https://sgmanager.ng.bluemix.net',
  'host':'dashdb-entry-yp-dal09-07.services.dal.bluemix.net',
  'username':'dash10135'
}


### Write Results

Now, select all the values we will use to build a ML model in Lab 3 and write to the DB2 database.  We will only load the Report Type, Category, Location and the Severity level columns into our new table in DB2 Warehouse.  We'll call the table "SUPPLYCHAIN_PREDICT."

In [30]:
valuesToWrite = modifiedsupplychain.select("Report_Type",  "Category", "Location", "Severity")
valuesToWrite.write.jdbc("jdbc:db2://dashdb-entry-yp-dal09-07.services.dal.bluemix.net:50000/BLUDB", "SUPPLYCHAIN_PREDICT", properties = {"user" : credentials_1["username"], "password" : credentials_1["password"]},mode="overwrite")

You should now have a new table in your DB2 Warehouse instance called "SUPPLYCHAIN_PREDICT." In Lab 3, we will export this table as a CSV and use it as a dataset to build our predictive model.

## Create Version

Save a new version of the notebook by selecting <b>File</b> > <b>Save Version</b>

![](https://raw.githubusercontent.com/jpatter/LMCO/master/Lab-1/images/FileOptions.PNG)

or by selecting the <b>Versions</b> icon and selecting <b>Save Version</b>.

![](https://raw.githubusercontent.com/jpatter/LMCO/master/Lab-1/images/versions-button.png)

You can have up to ten (10) versions of a notebook. Notebook versions are saved in a FIFO manner.


## Download notebook

Notebooks can be downloaded in notebook (.ipynb), Python (.py), HTML (.html), markdown (.md) or reST (.rst) format.  Use <b>File</b> > <b>Download as</b> to download the notebook in any of the formats.


![](https://raw.githubusercontent.com/jpatter/LMCO/master/Lab-1/images/FileOptions.PNG)

## Resources

Select the <b>Find Resources in the Community</b> link to display a search bar, documentation hotlinks, and a link to Stack Overflow's Data Science Experience section.

![](https://raw.githubusercontent.com/jpatter/LMCO/master/Lab-1/images/community-button.png)

![](https://raw.githubusercontent.com/jpatter/LMCO/master/Lab-1/images/Community-Resources.PNG)



## Get Help

Select the icon in the lower right corner

![](https://raw.githubusercontent.com/jpatter/DSX/master/Lab-1/images/Help-Icon.PNG)

to display a help window connecting to IBM DSX Support.

![](https://raw.githubusercontent.com/jpatter/DSX/master/Lab-1/images/Help-Window.PNG)

## Summary

In this lab, you learned how to establish a JDBC connection to a DB2 Warehouse database from Python, load data into data frames, perform exploratory analysis using Spark and finally write a modified dataset to a DB2 Warehouse database.

## Take it farther

### Free courses:

![](https://cognitiveclass.ai/courses/?utm_source=tutorial-dashdb-python&utm_medium=github&utm_campaign=bdu)

![](https://bigdatauniversity.com/courses/?utm_source=tutorial-dashdb-python&utm_medium=github&utm_campaign=bdu)

![](https://ibm.box.com/shared/static/xomeu7dacwufkoawbg3owc8wzuezltn6.png)